In [ ]:
'''
!!!!!!! Эта ячейка для запуска сборщика мозаик на Google Drive через Google Colab !!!!!!!

'''

from google.colab import drive
drive.mount('/content/drive')
# Определение директорий исходных изображений и назначения для мозаики
src_dir = '/content/drive/MyDrive/UII/Capture/'
dest_dir = '/content/drive/MyDrive/UII/Capture/Mosaic/'

In [1]:
'''
!!!!!!! Эта ячейка для запуска сборщика мозаик на локальной машине через Jupyter Notebook (или IDE) !!!!!!!

'''

# Определение директорий исходных изображений и назначения для мозаики
src_dir = r'C:\Users\anton\UII\AZavod_Ural'
dest_dir = r'C:\Users\anton\UII\AZavod_Ural\Mosaic'

In [7]:
'''
!!!!!!! Собственно сам сборщик мозаик !!!!!!!

'''

import os
import shutil
import random
from tqdm import tqdm
from PIL import Image

def delete_directory(path: str):
    # Удаляет директорию по заданному пути, если она существует
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
        except OSError as e:
            print(f'[ERROR] Error deleting a directory: `{path}`: {e.strerror}')

def ensure_directory_exists(directory):
    # Создает директорию, если она не существует
    if not os.path.exists(directory):
        os.makedirs(directory)

def create_mosaic(src_dir, dest_dir, canvas_size):
    # Загрузка всех изображений из исходной директории и их масштабирование
    all_images = []
    for file in os.listdir(src_dir):
        # Проверка, является ли файл изображением по его расширению
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')):
            path = os.path.join(src_dir, file)
            try:
                # Открытие и масштабирование изображения до размера полотна
                with Image.open(path) as img:
                    img.thumbnail((canvas_size, canvas_size), Image.Resampling.LANCZOS)
                    all_images.append(img.copy())
            except Exception as e:
                print(f"Ошибка при открытии изображения {file}: {e}")

    if not all_images:
        print("В папке нет изображений.")
        return

    # Инициализация прогресс-бара для отображения прогресса обработки
    cnt_img = len(all_images)
    with tqdm(total=cnt_img, desc="Обработка директории", unit=" files", leave=True) as pbar:
        while all_images:
            # Создание нового полотна
            canvas = Image.new('RGB', (canvas_size, canvas_size), (0, 0, 0))
            x, y = 0, 0
            next_row_y = 0
            filled_rows_width = []  # Для хранения ширины каждого заполненного ряда

            # Размещение изображений на текущем полотне
            while all_images:
                img = random.choice(all_images)
                # Размещение изображения, если оно помещается на полотно
                if x + img.width <= canvas_size and y + img.height <= canvas_size:
                    canvas.paste(img, (x, y))
                    x += img.width
                    filled_rows_width.append(x)  # Обновление ширины текущего ряда
                    next_row_y = max(next_row_y, y + img.height)
                    all_images.remove(img)  # Удаление использованного изображения из списка
                    pbar.update(1)
                else:
                    # Переход к новому ряду или полотну
                    if y + next_row_y >= canvas_size:  # Проверка на переход к новому полотну
                        break
                    x = 0
                    y = next_row_y

            # Дополнительный перебор 10% от оставшихся изображений
            extra_images_to_try = len(all_images) // 10
            while extra_images_to_try > 0 and all_images:
                img = random.choice(all_images)
                # Попытка разместить дополнительные изображения на полотне
                if x + img.width <= canvas_size and y + img.height <= canvas_size:
                    canvas.paste(img, (x, y))
                    x += img.width
                    filled_rows_width.append(x)  # Обновление ширины ряда
                    next_row_y = max(next_row_y, y + img.height)
                    all_images.remove(img)
                    pbar.update(1)
                extra_images_to_try -= 1

            # Расчет смещения для центровки мозаики на полотне
            max_filled_width = max(filled_rows_width, default=0)
            offset_x = (canvas_size - max_filled_width) // 2
            offset_y = (canvas_size - next_row_y) // 2
            final_canvas = Image.new('RGB', (canvas_size, canvas_size),(0, 0, 0))
            final_canvas.paste(canvas, (offset_x, offset_y))
            # Сохранение готового полотна в указанной директории
            final_canvas.save(os.path.join(dest_dir, f'mosaic_{random.randint(0, 10000)}.jpg'))

    # Вывод количества созданных полотен
    print(f"\nИсходные {cnt_img} картинок размещены на {len(os.listdir(dest_dir))} полотнах")

# Определение директорий исходных изображений и назначения для мозаики
#src_dir = r'C:\Users\anton\UII\AZavod_Ural'
#dest_dir = r'C:\Users\anton\UII\AZavod_Ural\Mosaic'
# Удаление предыдущей директории мозаики, если она существует
delete_directory(dest_dir)
# Создание новой директории для мозаики
ensure_directory_exists(dest_dir)
# Вызов функции создания мозаики
create_mosaic(src_dir, dest_dir, 640)

Обработка директории: 100%|██████████| 746/746 [00:07<00:00, 103.76 files/s]


Исходные 746 картинок размещены на 391 полотнах
